# Towards Quantum Measurement Error Mitigation

## How to connect CDR with quantum state tomography

Quantum computers are astonishing devices. Yet, they are error-prone, too. Therefore, we need to implement quantum error mitigation methods to reduce the negative effect of errors on our computation results. 

In a series of previous posts, we [learned the Clifford Data Regression method](https://pyqml.medium.com/mitigating-quantum-errors-using-clifford-data-regression-98ab663bf4c6) and mitigated errors in a [simulated environment](https://towardsdatascience.com/how-to-implement-quantum-error-mitigation-with-qiskit-and-mitiq-e2f6a933619c) and on a [real quantum computer](https://towardsdatascience.com/practical-error-mitigation-on-a-real-quantum-computer-41a99dddf740).

The results are encouraging. Yet, an unexpected obstacle appeared when I tried to use it to participate in IBM's Quantum Open Science Prize.

IBM asks us to simulate a three-particle Heisenberg Hamiltonian using Trotterization. No, that's not the problem. The problem is that they assess any submission through quantum state tomography. This is an approach to recreate a quantum state through measurements. More specifically, the problem is that they use the `StateTomographyFitter` inside Qiksit. This implementation builds upon experiment counts. But the CDR method works with expectation values.

Let me illustrate the problem a little bit. The following figure depicts the simple case of a 1-qubit quantum circuit.

![](assets/ev.png)

Whenever we look at a qubit, it is either 0 or 1. That's it. Which of both depends on chance and the internal quantum state. Say the qubit is in state $|+\rangle$. In this state, the measuring 0 and 1 have the same probability. In this state, measuring zero and one have the same probability. But, we can't conclude on the probabilities when running the quantum circuit only once.

But when we run it repeatedly, say a thousand times, we would see zero and one 500 times each, except for a slight statistical variance.

Unfortunately, current quantum computers are noisy devices. We sometimes measure a 0 for a 1 and vice versa. So, the results become blurred. For instance, we would measure 0 412 times. 

The raw result of running a quantum circuit is the measurement. Since we almost always execute circuits multiple times, we count the measurements. 

So, let's look at the source code in Qiskit of such a circuit.


In [1]:
from qiskit import QuantumCircuit, execute, Aer

# Create a quantum circuit with one qubit
qc = QuantumCircuit(1)  
qc.h(0)
qc.measure_all()

We define a quantum circuit with a single qubit and apply the Hadamard gate to it. This transformation gate puts the qubit (from the initial state $|0\rangle$) into state $|+\rangle$. At the end of the circuit, we measure the qubit. The following image shows the circuit diagram.

In [2]:
qc.draw(output='text')

┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0

Let's run this circuit 1,000 times. We do this by getting a `backend` (here, a noise-free statistical simulator) and calling the `execute` function.

In [3]:
# Tell Qiskit how to simulate our circuit
backend = Aer.get_backend('qasm_simulator') 

# Do the simulation, returning the result
result = execute(qc, backend, shots=1000).result()

We get back a Qiskit result object ([Qiskit reference](https://qiskit.org/documentation/stubs/qiskit.result.Result.html)). The essential function of this object is `get_counts` because it tells us what we saw when looking at the qubit.

In [4]:
result.get_counts()

{'1': 499, '0': 501}

It is a simple Python dictionary with the measurement result as the key and the number of times we observed this result as the value. It is now up to us to interpret these counts and do something meaningful. From now on, these results are as good as any other statistical data. We can use them to calculate further values, such as the expectation value. This is the probabilistic expected value of the measurement of an experiment. It is similar to a classical expectation value. For example, consider tossing a fair coin that lands on heads and tails equally likely. If you assign the value 1 to heads and 0 to tails, the expectation value is $0.5*1+0.5*0=0.5$. It is the same for a qubit in state $|+\rangle$.

Usually, when you see the calculation of expectation values in quantum computing, it looks like this $\langle \psi|Z|\psi\rangle$. The letter "psi" ($\psi$) denotes the quantum state, and the Z in the middle symbolizes the observable. In this case, it is the Z-axis. 

The important thing here is that this notation introduces the concept of an observable. When we measured our qubit in Qiskit before, we implicitly chose the Z-observable because this is the default measurement basis for a qubit in Qiskit. So, we are essentially talking about the same concept. In [this post](https://towardsdatascience.com/how-to-implement-quantum-error-mitigation-with-qiskit-and-mitiq-e2f6a933619c), we looked at observables in more detail. The one thing that we need to know is that there is not a single observable, but many. Just think about a sphere like the earth. The observable is the specific point from which you look at the globe. The world looks different depending on the perspective, yet, it is the same.

![](assets/earth.png)

Essentially, the counts and the expectation value are closely tight together. They both have their uses. While the counts contain more information about the different measurements, the expectation value is better to work with because it is a single number.

This is the point of struggle I described in my previous post. While the quantum error mitigation method CDR uses the simplicity of the expectation value, the quantum state tomography that IBM uses to evaluate the performance of the error mitigation works with the counts.

To participate in IBM's challenge, it is now our job to integrate the two.

I chose to adapt the CDR to work with an unchanged state tomography because the latter is IBM's evaluation method. I believe messing around with their assessment tool may disqualify myself from the contest.

So, we need to change the CDR method to change a counts dictionary instead of a single number.

Let's revisit the CDR briefly.

![](assets/cdr_steps.png)


The CDR method has three steps. First, we generate training data. Then, in this step, we run our quantum circuit twice. Once on a classical computer to obtain the exact value of an observable's expectation value of interest. And once on a real quantum computer to produce the noisy value.

In the second step, we create a linear model of the relationship between the noisy and the exact values. Building a linear model from a set of data points is known as regression. Therefore the name CDR--Clifford Data Regression.

Finally, we use the model to transform a noisy expectation value into a mitigated one by predicting the noise-free value.

All these steps need to work with the Qiskit [experiment result](https://qiskit.org/documentation/stubs/qiskit.result.Result.html). However, the problem is that this is an object that the Qiskit `execute` function creates. It stores most of its data in a read-only manner that we can't change any more.

But, we can apply a little trick. We write our own `Result` class that allows us to change the counts afterward.

Conceptually, we create a new Python class that serves the one function the state tomography uses. This is the `get_counts` function. So, when the state tomography function queries the counts, it gets a response. But since we implement this new class, we can also provide a function that overwrites the counts.

The following listing depicts the source code of our `OwnResult` class.


In [32]:
from qiskit.result import Result

from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit.exceptions import QiskitError
from qiskit.result.counts import Counts

class OwnResult(Result):
    
    def __init__(self, result):
        self._result = result
        self._counts = {}
            
        
    def get_counts(self, experiment=None):

        if experiment is None:
            exp_keys = range(len(self._result.results))
        else:
            exp_keys = [experiment]
        

        dict_list = []
        for key in exp_keys:
            exp = self._result._get_experiment(key)
            try:
                header = exp.header.to_dict()
            except (AttributeError, QiskitError):  # header is not available
                header = None

            if "counts" in self._result.data(key).keys():
                if header:
                    counts_header = {
                        k: v
                        for k, v in header.items()
                        if k in {"time_taken", "creg_sizes", "memory_slots"}
                    }
                else:
                    counts_header = {}
                    
                    
                # CUSTOM CODE STARTS HERE #######################
                dict_list.append(Counts(
                    self._counts[str(key)] if str(key) in map(lambda k: str(k), self._counts.keys()) else self._result.data(key)["counts"]
                    , **counts_header))
                # CUSTOM CODE ENDS HERE #######################
                
            elif "statevector" in self._result.data(key).keys():
                vec = postprocess.format_statevector(self._result.data(key)["statevector"])
                dict_list.append(statevector.Statevector(vec).probabilities_dict(decimals=15))
            else:
                raise QiskitError(f'No counts for experiment "{repr(key)}"')

        # Return first item of dict_list if size is 1
        if len(dict_list) == 1:
            return dict_list[0]
        else:
            return dict_list
        
        
    def set_counts(self, counts, experiment=None):
        self._counts[str(experiment) if experiment is not None else "0"] = counts

The class takes the existing Qiskit result as an initialization parameter. Further, we specify `_counts` as a member variable that we initialize with an empty dictionary. This will hold our changed counts. The code of the `get_count` function is copied from the original source code except for two little thing. First, whenever we refer to the a property of the result, such as `data` we need to look into `self._result.data` instead of `self.data`. Second, at lines 40-44, we look into the custom member function for the actual counts. If they exist (`if str(key) in self._counts.keys()if str(key) in self._counts.keys()`) we return the changed counts (`self._counts[str(key)]self._counts[str(key)]`). If they don't exist, we return the original counts (`self._result.data(key)["counts"]`)

Let's have a look at how that works.

In [35]:
# create our own Result object
own = OwnResult(result)
print("original result: ", own.get_counts())

# set new counts
own.set_counts({"0": 100, "1": 900})
print("Changed counts:  ", own.get_counts())

original result:  {'1': 499, '0': 501}
Changed counts:   {'0': 100, '1': 900}


So, let's see whether we can use our `OwnResult` inside the state tomography.

In the following snippet, we create the same simple circuit we used before. The only difference is that we omit the measurement because we need to create the `state_tomography_circuits` from it. Then, we run these circuits on a noise-free `Qasm-Simulator` and store the result of it.

So, now we're ready for the interesting part. We loop through the circuits in the list of experiments (`st_qcs`). For each of the circuits, we set the counts to a fixed dictionary with arbitrary values. We don't care about the values for now because we only want to verify whether the `StateTomographyFitter` works with our `OwnResult`.

Finally, we compute the fidelity based on the original and the changed results.

In [52]:
# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
from qiskit.opflow import Zero, One
from qiskit.providers.aer import QasmSimulator
from qiskit import execute, transpile, assemble

# create a circuit without measurement
qc = QuantumCircuit(1)  
qc.h(0)

# The expected final state; necessary to determine state tomography fidelity
target_state = (One).to_matrix()  # |1>

# create the three state tomography circuits
st_qcs = state_tomography_circuits(qc, [0])

# Noiseless simulated backend
sim = QasmSimulator()

job = execute(st_qcs, sim, shots=1000)

# get the result
result = job.result()

# put the result in our own class
own_res = OwnResult(result)

# loop through the experiments
for experiment in st_qcs:
    exp_keys = [experiment]
    
    for key in exp_keys:
        exp = result._get_experiment(key)        
        own_res.set_counts({"0": 100, "1": 900}, key)


# Compute fidelity with original result
orig_tomo_fitter = StateTomographyFitter(result, st_qcs)
orig_rho_fit = orig_tomo_fitter.fit(method='lstsq')
orig_fid = state_fidelity(orig_rho_fit, target_state)
print("original fidelity:", orig_fid)


# Compute fidelity with chnaged result
tomo_fitter = StateTomographyFitter(own_res, st_qcs)
rho_fit = tomo_fitter.fit(method='lstsq')
fid = state_fidelity(rho_fit, target_state)
print("changed fidelity: ", fid)

original fidelity: 0.49800015998080377
changed fidelity:  0.7886751345948128


The output shows that the fidelity we calculated based on the changed counts differs significantly from the one we calculated based on the original counts. Apparently, the `StateTomographyFitter` works with our custom counts. This creates the prerequisite for mitigating errors in the next step.